## Install Tensortrade

In [104]:
#!pip install git+https://github.com/kodiakcrypto/tensortrade.git#egg=tensortrade[data1]
#!pip install git+https://github.com/kodiakcrypto/tensortrade.git#egg=tensortrade[data2]

In [105]:
#!pip install ta newtulipy

In [106]:
import tulipy as _ta
import ta

## Get TA Data Columns

In [107]:
def XcrossY(candles, sma, sma_length):
    arr = []
    for i in range(len(sma) -1 ):
        if i <= sma_length: arr.append(0)
        else:
            if candles['high'].iloc[i-1] <= sma[i-1] and candles['high'].iloc[i] > sma[i]:
                arr.append(1)
            elif candles['low'].iloc[i-1] >= sma[i-1] and candles['low'].iloc[i] < sma[i]:
                arr.append(-1)
            else:
                arr.append(0)
    return arr

def slopeOfSeries(values, lookback):
    """get rise over run"""
    arr = []
    for i in range(len(values)-1):
        if i <= lookback: arr.append(0)
        else:
            slope = (values[i] - values[i-lookback]) / lookback
            if   slope > 0: arr.append(1)
            elif slope < 0: arr.append(-1)
            else:           arr.append(0)
    return arr

def checkForConsecutiveCandles(candles):
    """ 3 green and not 4"""
    arr = []
    for i in range(len(candles)-1):
        if i <= 3: arr.append(0)
        else:
            if (candles['close'].iloc[i] > candles['open'].iloc[i] and
                candles['close'].iloc[i-1] > candles['open'].iloc[i-1] and
                candles['close'].iloc[i-2] > candles['open'].iloc[i-2] and
                candles['close'].iloc[i-3] <= candles['open'].iloc[i-3]):
                arr.append(1)
            #3 red not 4
            elif (candles['close'].iloc[i] < candles['open'].iloc[i] and
                candles['close'].iloc[i-1] < candles['open'].iloc[i-1] and
                candles['close'].iloc[i-2] < candles['open'].iloc[i-2] and
                candles['close'].iloc[i-3] >= candles['open'].iloc[i-3]):
                arr.append(-1)
            else:
                arr.append(0)
    return arr

def getBBvsPrice(close_array, candles, stddev):
    arr = []
    period=20
    (bbands_lower, _, bbands_upper) = _ta.bbands(close_array, period=period, stddev=stddev)
    for i in range(len(bbands_upper)-1):
        if i <= period:
            arr.append(0)
        if candles['high'].iloc[i] > bbands_upper[i] and candles['low'].iloc[i] < bbands_lower[i]:
            arr.append(0)
        elif candles['high'].iloc[i] > bbands_upper[i]:
            arr.append(1)
        elif candles['low'].iloc[i] < bbands_lower[i]:
            arr.append(-1)
        else:
            arr.append(0)
    return arr

def getSMACrosses(sma_fast, sma_slow):
    arr = []
    for i in range(len(sma_slow)-2):
        if sma_fast[i] <= sma_slow[i] and sma_fast[i+1] > sma_slow[i+1]:
            arr.append(1)
        elif sma_fast[i] >= sma_slow[i] and sma_fast[i+1] < sma_slow[i+1]:
            arr.append(-1)
        else:
            arr.append(0)
    return arr

def sauceCity(candles, date_col_name='date',
                       open_col_name='open',
                       high_col_name='high',
                       low_col_name='low',
                       close_col_name='close'):
    lookback          = 20
    fast_sma_length   = 36
    slow_sma_length   = 85
    filter_sma_length = 200

    timestamps = candles[date_col_name].to_numpy()
    close_array = candles[close_col_name].to_numpy()

    # BBANDS
    price_vs_bbands_2x = getBBvsPrice(close_array, candles, stddev=2)
    price_vs_bbands_3x = getBBvsPrice(close_array, candles, stddev=3)

    # Get price vs SMAs and SMA crosses
    sma_fast = _ta.sma(close_array, period=fast_sma_length)
    sma_slow = _ta.sma(close_array, period=slow_sma_length)
    price_vs_fastSMA = XcrossY(candles, sma_fast, fast_sma_length)
    price_vs_slowSMA = XcrossY(candles, sma_slow, slow_sma_length)
    sma_crosses = getSMACrosses(sma_fast, sma_slow)

    # get SMA 200 vals + slope
    sma_filter = _ta.sma(close_array, filter_sma_length)
    sma_filter_slope = slopeOfSeries(sma_filter, lookback)
    price_vs_filterSMA = XcrossY(candles, sma_filter, filter_sma_length)

    # check for 3 consecutive same color candles
    consecutive_candles = checkForConsecutiveCandles(candles)

    #trendline crosses
    #lookbacks

    print(
        len(timestamps),
        len(price_vs_bbands_2x),
        len(price_vs_bbands_3x),
        len(price_vs_fastSMA),
        len(price_vs_slowSMA),
        len(sma_crosses),
        len(sma_filter_slope),
        len(price_vs_filterSMA),
        len(consecutive_candles)
    )

    copy_arr = [price_vs_bbands_2x.copy(),price_vs_bbands_3x.copy(),
                price_vs_fastSMA.copy(),price_vs_slowSMA.copy(),sma_crosses.copy(),
                sma_filter_slope.copy(),price_vs_filterSMA.copy(),consecutive_candles.copy()]

    for i in range(len(copy_arr)):
        while len(copy_arr[i]) != len(timestamps):
            if len(copy_arr[i]) < len(timestamps):
                copy_arr[i] = np.insert(copy_arr[i], 0, 0)
            elif len(copy_arr[i]) > len(timestamps):
                copy_arr[i] = copy_arr[i][1:]

    price_vs_bbands_2x = copy_arr[0]
    price_vs_bbands_3x = copy_arr[1]
    price_vs_fastSMA = copy_arr[2]
    price_vs_slowSMA = copy_arr[3]
    sma_crosses = copy_arr[4]
    sma_filter_slope = copy_arr[5]
    price_vs_filterSMA = copy_arr[6]
    consecutive_candles = copy_arr[7]

    print(
        len(timestamps),
        len(price_vs_bbands_2x),
        len(price_vs_bbands_3x),
        len(price_vs_fastSMA),
        len(price_vs_slowSMA),
        len(sma_crosses),
        len(sma_filter_slope),
        len(price_vs_filterSMA),
        len(consecutive_candles)
    )
    all_data = list(zip(
        timestamps,
        price_vs_bbands_2x,price_vs_bbands_3x,
        price_vs_fastSMA,price_vs_slowSMA,sma_crosses,
        sma_filter_slope,price_vs_filterSMA,consecutive_candles,
    ))

    all_data = pd.DataFrame(all_data, columns=[
        'date','price_vs_bbands_2x','price_vs_bbands_3x',
        'price_vs_fastSMA','price_vs_slowSMA','sma_crosses',
        'sma_filter_slope','price_vs_filterSMA','consecutive_candles'])

    return all_data

## Import Needed Tensortrade modules

In [108]:
# À installer si ce n’est pas encore fait :
# !pip install git+https://github.com/kodiakcrypto/tensortrade.git#egg=tensortrade[data]

from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.services.execution.simulated import execute_order

from tensortrade.feed.core import Stream, DataFeed  # ✅ Pas besoin de Module

from tensortrade.env.default import create as create_environment
#from tensortrade.env.default.render import PlotlyTradingChart

from tensortrade.agents import DQNAgent


## Fetch Candle data

In [110]:
import ccxt
import pandas as pd

# Paramètres
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1d'

# Télécharger les bougies OHLCV
ohlcv = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe)

# Construire un DataFrame
df_BTC = pd.DataFrame(ohlcv, columns=["Date", "BTC:open", "BTC:high", "BTC:low", "BTC:close", "BTC:volume"])

# Convertir l'horodatage en datetime
df_BTC["Date"] = pd.to_datetime(df_BTC["Date"], unit="ms")

# Fixer la colonne Date comme index (facultatif)
df_BTC.set_index("Date", inplace=False).head()


,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
Date,,,,,
2024-01-18,42776.09,42930.00,40683.28,41327.50,43907.51641
2024-01-19,41327.51,42196.86,40280.00,41659.03,48342.74559
2024-01-20,41659.03,41872.56,41456.30,41696.04,15923.99493
2024-01-21,41696.05,41881.39,41500.98,41580.33,11730.16301
2024-01-22,41580.32,41689.65,39431.58,39568.02,55426.19911


In [111]:
import ccxt
import pandas as pd

exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1d'

# Téléchargement brut via CCXT
ohlcv = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe)

# Conversion en DataFrame
df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
df.set_index("timestamp", inplace=True)

df.head()

,open,high,low,close,volume
timestamp,,,,,
2024-01-18,42776.09,42930.00,40683.28,41327.50,43907.51641
2024-01-19,41327.51,42196.86,40280.00,41659.03,48342.74559
2024-01-20,41659.03,41872.56,41456.30,41696.04,15923.99493
2024-01-21,41696.05,41881.39,41500.98,41580.33,11730.16301
2024-01-22,41580.32,41689.65,39431.58,39568.02,55426.19911


## Get Ta Vals

In [112]:
#OHCLV dataframe for plotly chart w.o indicators
ohlcv_df = df_BTC.reset_index().rename({"Date": "datetime"}, axis='columns')

In [114]:
import numpy as np

ta_df = df_BTC.copy()
ta_df.columns = [name.lower().split(':')[1] for name in df.columns]

ta_df = sauceCity(ta_df.reset_index().rename({"Date": "date"}, axis='columns'))

ta_df['close'] = df['BTC:close'].to_numpy()
ta_df['open_minus_close'] = np.log(ta_df['open']) - np.log(ta_df['close'])
ta_df['high_minus_low'] = np.log(ta_df['high']) - np.log(ta_df['low'])

ta_df = ta_df.set_index('date')
ta_df

IndexError: list index out of range

In [ ]:
df = ta_df

In [ ]:

df['obv'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close']).average_true_range()
df['bb_upper_cross'] = ta.volatility.BollingerBands(df['close']).bollinger_hband_indicator()
df['bb_lower_cross'] = ta.volatility.BollingerBands(df['close']).bollinger_lband_indicator()
df['macd'] = ta.trend.MACD(df['close']).macd()
df['macd_signal'] = ta.trend.MACD(df['close']).macd_signal()

In [ ]:
# MACD > or < 0.0
i = 0
test = []
for y in df.macd.values:
    if y > 0.0:
        if df.macd_signal.values[i] > 0.0 or i != 0:
            test.append(1)
        else:
            test.append(test[-1])
    elif y < 0.0:
        if df.macd_signal.values[i] < 0.0 or i != 0:
            test.append(-1)
        else:
            test.append(test[-1])
    i += 1

In [ ]:
df['macd_above_zero'] = pd.Series(test)
df['pre_macd_above_zero'] = df['macd_above_zero'].shift(1)
df['macd_cross_zero'] = np.where(df['macd_above_zero'] == df['pre_macd_above_zero'], 0, 1)
df = df.drop(['macd', 'macd_signal', 'pre_macd_above_zero', 'macd_above_zero'], axis=1)
df['macd_cross_zero'][0] = 0

df = df.set_index("date")

base, quote = symbol.split('_')
df.columns = [base + ":" + name.lower() for name in df.columns]

## Feature correlation analysis



### Create correlation matrix


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Display correlation heatmap

In [ ]:
plt.figure(figsize=(17,17))
corr_matrix = ta.add_all_ta_features(df, 'BTC:open', 'BTC:high', 'BTC:low', 'BTC:close', 'BTC:volume').corr().abs()
sns.heatmap(corr_matrix, annot=False, linewidth=0.3, center=0, cmap='Blues', vmin=-1)

In [ ]:
corr_matrix

### Identify Highly Correlated Features

In [ ]:
import numpy as np

#apply log diff pricing to OHLC columns
for column in ['BTC:open','BTC:high','BTC:low','BTC:close']:
  if column in df.columns:
      df[column] = np.log(df[column]).diff()

print(df) # Before dropping columns for comparison

# Slice upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find column names of features with correlation greater than 0.95
# < 0.0 gets rid of data that has negative value despite .abs()
to_drop = []
for column in upper.columns:
    if ((not column[-4:].lower() == 'open' and
         not column[-4:].lower() == 'high' and
         not column[-3:].lower() == 'low' and
         not column[-5:].lower() == 'close') and
        any(upper[column] > 0.95) or
        any(upper[column] < 0.0)):
        to_drop.append(column)

# Drop those columns
#df = df.drop(df[to_drop], axis=1)
#print(df)

# Idea
# Add in MA's of the rejected types with new periods and correlate again
# TA_Features.add_moving_avgs(indicators: list = 'all'):
#    list of MA funcs = []
# for func in MA_func_list:
#    func(data['BTC:close'])

## Correlation Work

In [ ]:
c1 = corr_matrix[[x for x in corr_matrix.columns if x.startswith('volume')]].drop(
                [x for x in corr_matrix.index if not x.startswith('volume')])
c1 = c1.drop(
                ['volume_nvi', 'volume_adi', 'volume_fi', 'volume_vwap',
                 'volume_cmf', 'volume_em', 'volume_sma_em', 'volume_vpt']
            ).drop(
                ['volume_nvi', 'volume_adi', 'volume_fi', 'volume_vwap',
                 'volume_cmf', 'volume_em', 'volume_sma_em', 'volume_vpt'],
            axis=1)
c1

In [ ]:
c2 = corr_matrix[[x for x in corr_matrix.columns if x.startswith('momentum')]].drop(
                [x for x in corr_matrix.index if not x.startswith('momentum')],axis=0)
c2 = c2.drop(
                ['momentum_rsi', 'momentum_stoch', 'momentum_wr', 'momentum_roc',
                 'momentum_ao', 'momentum_uo', 'momentum_stoch_signal', 'momentum_tsi',
                 'momentum_kama', 'momentum_mfi']
            ).drop(
                ['momentum_rsi', 'momentum_stoch', 'momentum_wr', 'momentum_roc',
                 'momentum_ao', 'momentum_uo', 'momentum_stoch_signal', 'momentum_tsi',
                 'momentum_kama', 'momentum_mfi'],
            axis=1)
c2

In [ ]:
c3 = corr_matrix[[x for x in corr_matrix.columns if x.startswith('other')]].drop(
                [x for x in corr_matrix.index if not x.startswith('other')],axis=0)
c3
# self._dr = np.log(self._close).diff()
# self._dr *= 100

In [ ]:
c4 = corr_matrix[[x for x in corr_matrix.columns if x.startswith('vola')]].drop(
                [x for x in corr_matrix.index if not x.startswith('vola')],axis=0)
c4 = c4.drop(
                ['volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbp',
                 'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
                 'volatility_kcp', 'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
                 'volatility_dch', 'volatility_bbw']
             ).drop(
                ['volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbp',
                 'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
                 'volatility_kcp', 'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
                 'volatility_dch', 'volatility_bbw'],
             axis=1)
c4

In [ ]:
c5 = corr_matrix[[x for x in corr_matrix.columns if x.startswith('trend')]].drop(
                [x for x in corr_matrix.index if not x.startswith('trend')],axis=0)
c5 = c5.drop(
                ['trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
                 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
                 'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
                 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst', 'trend_kst_sig',
                 'trend_kst_diff', 'trend_visual_ichimoku_a', 'trend_visual_ichimoku_b',
                 'trend_aroon_up', 'trend_aroon_down', 'trend_aroon_ind', 'trend_psar_up',
                 'trend_psar_down', 'trend_psar_up_indicator', 'trend_psar_down_indicator',
                 'trend_ichimoku_conv', 'trend_ichimoku_base', 'trend_ichimoku_a',
                 'trend_ichimoku_b', 'trend_macd_diff']
             ).drop(
                ['trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
                 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
                 'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
                 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst', 'trend_kst_sig',
                 'trend_kst_diff', 'trend_visual_ichimoku_a', 'trend_visual_ichimoku_b',
                 'trend_aroon_up', 'trend_aroon_down', 'trend_aroon_ind', 'trend_psar_up',
                 'trend_psar_down', 'trend_psar_up_indicator', 'trend_psar_down_indicator',
                 'trend_ichimoku_conv', 'trend_ichimoku_base', 'trend_ichimoku_a',
                 'trend_ichimoku_b', 'trend_macd_diff'],
             axis=1)
c5
# trend_macd > 0
# and trend_macd_signal > 0
# and trend_macd_signal > trend_macd

In [ ]:
d = []
d.extend(c1.columns)
d.extend(c2.columns)
d.extend(c4.columns)
d.extend(c5.columns)

corr_matrix = df[d].corr().abs()
corr_matrix

#get a bbgrid going on many timedrames so actions
#can be taken on lower ones and the higher ones be the data

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

upper[[x for x in upper.columns if any(upper[x] > 0.5)]]

## Enable Plotly Chart

In [ ]:
# ipywidgets is required to run Plotly in Jupyter Notebook.
!pip install ipywidgets
import ipywidgets

from tensortrade.environments.render import PlotlyTradingChart
# from tensortrade.environments.render import FileLogger

import plotly.io as pio
pio.renderers
pio.renderers.default = "colab"

chart_renderer = PlotlyTradingChart(
    display=True,  # show the chart on screen (default)
    height=800,  # affects both displayed and saved file height. None for 100% height.
    save_format='html',  # save the chart to an HTML file
    # auto_open_html=True,  # open the saved HTML chart in a new browser tab
)

# file_logger = FileLogger(
#     filename='example.log',  # omit or None for automatic file name
#     path=None  # create a new directory if doesn't exist, None for no directory
# )

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 10.5 MB/s 


##Optuna

In [ ]:
!pip install optuna==1.2.0 # DB was created in 1.2, 1.4 dont like that
import optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from tensortrade.agents import DQNAgent

def Train_DQN_Agent():
    n_episodes =  10000
    window_size = 1
    learning_rate = 0.001
    discount_factor = 0.9999

    # Random slice of data for training
    # start_bound = len(df['close']) / 100
    # start_slice = 0

    # end_bound_2 = start_slice + 100#len(df['BTC:close'])
    # end_bound_1 = start_slice + window_size * 2
    # if end_bound_1 >= end_bound_2:
    #     end_bound_1 = end_bound_2 - window_size
    # end_slice = start_slice + 100


    batch_size = 128
    memory_capacity = 1000
    eps_start = 0.9
    eps_end = 0.05
    eps_decay_steps = 200

    save_path = "/content/drive/MyDrive/agents/"
    n_steps = len(df['close']-1)#end_slice - start_slice + 1 .iloc[start_slice:end_slice]
    render_interval = 1
    # print('steps', end_slice - start_slice)

    save_every = 500
    update_target_every = 1000

    print(df.columns)
    with Module("TEST1") as moduleNodes:
        nodes = [Stream(name, list(df[name])) for name in df.columns]

    print(len(moduleNodes.inputs))
    feed = DataFeed([moduleNodes])
    print(feed._repr_())

    exchange = Exchange("TEST1", service=execute_order)(
            Stream("USD-BTC", list(df['close']))
    )
    portfolio = Portfolio(USD, [
        Wallet(exchange, 1000000 * USD),
        Wallet(exchange, 0 * BTC)
    ])

    env = TradingEnvironment(
        feed = feed,
        portfolio = portfolio,
        action_scheme = 'simple',
        window_size = window_size,
        reward_scheme = 'simple',
        enable_logger = True,
        renderers = [chart_renderer, 'screenlog'],
        price_history = ohlcv_df
    )
    print(env)

    agent = DQNAgent(env)

    mean_reward = agent.train(
        n_steps = n_steps,
        n_episodes = n_episodes,
        learning_rate = learning_rate,
        save_path = save_path,

        batch_size = batch_size,
        discount_factor = discount_factor,

        #save_every = save_every,
        update_target_every = update_target_every,
        memory_capacity = memory_capacity,
        render_interval = render_interval,

        eps_start = eps_start,
        eps_end = eps_end,
        eps_decay_steps = eps_decay_steps
    )

    chart_renderer.fig.show()

    return mean_reward

In [ ]:
Train_DQN_Agent()

Index(['price_vs_bbands_2x', 'price_vs_bbands_3x', 'price_vs_fastSMA',
       'price_vs_slowSMA', 'sma_crosses', 'sma_filter_slope',
       'price_vs_filterSMA', 'consecutive_candles', 'close'],
      dtype='object')
0
<Node: name=node:72, type=datafeed>


IndexError: ignored

In [ ]:

    # from tensortrade.actions import ManagedRiskOrders
    # action_scheme = ManagedRiskOrders(
    #     #durations=[6],
    #     stop_loss_percentages=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
    #                            0.005, 0.0075, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065],
    #     take_profit_percentages=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
    #                            0.005, 0.0075, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065],
    #     trade_sizes = [1/30, 1/25, 1/20, 1/15])
    # """
    # Args:
    #     pairs: A list of trading pairs to select from when submitting an order.
    #            e.g. TradingPair(BTC, USD)
    #     stop_loss_percentages: A list of possible stop loss percentages for each order.
    #     take_profit_percentages: A list of possible take profit percentages for each order.
    #     trade_sizes: A list of trade sizes to select from when submitting an order.
    #                  e.g. '[1, 1/3]' = 100% or 33% of balance is tradable.
    #                       '4' = 25%, 50%, 75%, or 100% of balance is tradable.

    #     order_listener (optional): An optional listener for order events executed by this action scheme.
    # """

    # from tensortrade.rewards import RiskAdjustedReturns
    # reward_scheme = RiskAdjustedReturns(
    #     return_algorithm='sortino',
    #     minimize_trades = True,
    #     window_size = window_size
    #     )
    # """
    #     Args:
    #         return_algorithm (optional): The risk-adjusted return metric to use.
    #                                      Options are 'sharpe' and 'sortino'. Defaults to 'sharpe'.
    #         risk_free_rate (optional): The risk free rate of returns to use for calculating metrics.
    #                                    Defaults to 0.
    #         target_returns (optional): The target returns per period for use in calculating the
    #                                    sortino ratio. Default to 0.
    # """


### Train DQN Agent with Optuna

In [ ]:
from tensortrade.agents import DQNAgent

def Train_DQN_Agent(trial):

    # OPTUNA PARAMETER TYPES
    # optimizer = trial.suggest_categorical('optimizer', ['MomentumSGD', 'Adam'])
    # num_layers = trial.suggest_int('num_layers', 1, 3)
    # dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)
    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    # drop_path_rate = trial.suggest_discrete_uniform('drop_path_rate', 0.0, 1.0, 0.1) #0.1 increments

    #set optunable vars
    #n_steps = trial.suggest_int('n_steps', 16, 100)
    n_episodes = trial.suggest_int('n_episodes', 1000, 10000)
    window_size = trial.suggest_int('window_size', 2, 100)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0001, 1.00)
    discount_factor = trial.suggest_uniform('discount_factor', 0.5000, 0.9999)

    # Random slice of data for training
    start_bound = len(df['close']) / 100
    start_slice = trial.suggest_int('start_slice', 0, start_bound)

    end_bound_2 = start_slice + 100#len(df['BTC:close'])
    end_bound_1 = start_slice + window_size * 2
    if end_bound_1 >= end_bound_2:
        end_bound_1 = end_bound_2 - window_size
    end_slice = trial.suggest_int('end_slice', end_bound_1, end_bound_2)


    batch_size = 128
    memory_capacity = 1000
    eps_start = 0.9
    eps_end = 0.05
    eps_decay_steps = 200
    save_path = "agents/"
    n_steps = end_slice - start_slice + 1
    render_interval = 10#int(n_steps / 5)
    print('steps', end_slice - start_slice)
    #save_every = int(n_episodes / 10)
    update_target_every = 1000


    with Module("coinbase") as coinbase_ns:
        nodes = [Stream(name, list(df[name][start_slice:end_slice])) for name in df.columns]

    feed = DataFeed([coinbase_ns])

    exchange = Exchange("coinbase", service=execute_order)(
            Stream("USD-BTC", list(df['close'][start_slice:end_slice]))
    )
    portfolio_list.append(Portfolio(USD, [
        Wallet(exchange, 1000000 * USD),
        Wallet(exchange, 0 * BTC)
    ]))


    from tensortrade.actions import ManagedRiskOrders
    action_scheme = ManagedRiskOrders(
        #durations=[6],
        stop_loss_percentages=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                               0.005, 0.0075, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065],
        take_profit_percentages=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                               0.005, 0.0075, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065],
        trade_sizes = [1/30, 1/25, 1/20, 1/15])
    """
    Args:
        pairs: A list of trading pairs to select from when submitting an order.
               e.g. TradingPair(BTC, USD)
        stop_loss_percentages: A list of possible stop loss percentages for each order.
        take_profit_percentages: A list of possible take profit percentages for each order.
        trade_sizes: A list of trade sizes to select from when submitting an order.
                     e.g. '[1, 1/3]' = 100% or 33% of balance is tradable.
                          '4' = 25%, 50%, 75%, or 100% of balance is tradable.

        order_listener (optional): An optional listener for order events executed by this action scheme.
    """

    from tensortrade.rewards import RiskAdjustedReturns
    reward_scheme = RiskAdjustedReturns(
        return_algorithm='sortino',
        minimize_trades = True,
        window_size = window_size
        )
    """
        Args:
            return_algorithm (optional): The risk-adjusted return metric to use.
                                         Options are 'sharpe' and 'sortino'. Defaults to 'sharpe'.
            risk_free_rate (optional): The risk free rate of returns to use for calculating metrics.
                                       Defaults to 0.
            target_returns (optional): The target returns per period for use in calculating the
                                       sortino ratio. Default to 0.
    """


    env = TradingEnvironment(
        feed = feed,
        portfolio = portfolio_list[-1],
        action_scheme = action_scheme,
        window_size = window_size,
        reward_scheme = reward_scheme,
        # enable_logger = True,
        renderers = [chart_renderer, 'screenlog'],
        price_history = ohlcv_df
    )
    env_list.append(env)

    agent = DQNAgent(env_list[-1])

    mean_reward = agent.train(
        n_steps = n_steps,
        n_episodes = n_episodes,
        learning_rate = learning_rate,
        save_path = save_path,

        batch_size = batch_size,
        discount_factor = discount_factor,

        #save_every = save_every,
        update_target_every = update_target_every,
        memory_capacity = memory_capacity,
        render_interval = render_interval,

        eps_start = eps_start,
        eps_end = eps_end,
        eps_decay_steps = eps_decay_steps
    )

    chart_renderer.fig.show()

    return mean_reward

### Shap - WORK IN PROGRESS

In [ ]:
!pip install shap
import shap

In [ ]:
from tensortrade.agents import DQNAgent
from tensortrade.actions import ManagedRiskOrders
from tensortrade.rewards import RiskAdjustedReturns

import tensorflow as tf


with Module("coinbase") as coinbase_ns:
    nodes = [Stream(name, list(df['BTC:close'])) for name in df.columns]

feed = DataFeed([coinbase_ns])

exchange = Exchange("coinbase", service=execute_order)(
        Stream("USD-BTC", list(df['BTC:close']))
)

portfolio = Portfolio(USD, [
    Wallet(exchange, 1000000 * USD),
    Wallet(exchange, 0 * BTC)
])

action_scheme = ManagedRiskOrders(
    stop_loss_percentages=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                            0.005, 0.0075, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065],
    take_profit_percentages=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                            0.005, 0.0075, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065],
    trade_sizes = [1/30, 1/25, 1/20, 1/15])

reward_scheme = RiskAdjustedReturns(
    return_algorithm='sortino',
    minimize_trades = True,
    window_size = 48
    )

env = TradingEnvironment(
    feed = feed,
    portfolio = portfolio,
    action_scheme = action_scheme,
    window_size = 48,
    reward_scheme = reward_scheme,
)

policy_network = None
try:
  policy_network = tf.keras.models.load_model(
      'agents/policy_network__4f977c8a-aefe-46db-9df1-15444dae5418__465.hdf5'
  )
except Exception as e:
  print(e)

agent = DQNAgent(env, policy_network=policy_network)

background = np.resize(np.array(df[['BTC:close']].sample(n=100)), (100, 1, 0))
print(background.shape)

In [ ]:
e = shap.DeepExplainer(agent.policy_network, background)
shap_values = e.shap_values(data[1:5])

In [ ]:
shap.image_plot(shap_values, -data[1:5])

###Run Optuna Trials
- each trial trains a network
- then at the end it tells us what one did best

In [ ]:
# Initiate Optuna trials
env_list = []
portfolio_list = []
n_trials = 1

# example callback (unused but passes out the 'portfolio'
# callbacks = [
#     lambda study_, frozen_trial_ : portfolio_chart(study_, frozen_trial_, portfolio)
# ]

study_name = 'new_921'

study = optuna.create_study(study_name=study_name,
                            storage=f'sqlite:///{study_name}.db',
                            load_if_exists=True,
                            direction='maximize')

study.optimize(Train_DQN_Agent,
               n_trials=n_trials)
               #callbacks=callbacks)

### Print results

In [ ]:
study.trials

In [ ]:
study.best_trial

In [ ]:
study.best_params

In [ ]:
study.best_value

In [ ]:
for portfolio in portfolio_list:
    portfolio.performance.plot(legend='reverse')

In [ ]:
for portfolio in portfolio_list:
    portfolio.performance.net_worth.plot()

In [ ]:
study_name = '1h_bb_macd_atr_obv_7'
study = optuna.load_study(study_name=study_name,
                          storage=f'sqlite:///{study_name}.db')

#Show all trials in DB
df = study.trials_dataframe(attrs=('value', 'params'))
df2 = pd.concat([df['value'],
                 df['params_learning_rate'],
                 df['params_n_episodes'],
                 df['params_window_size']
                 ],
                axis=1)
df2 = df2.sort_values('value', ascending=True)
print(df2.tail(20))

### Delete SQL entries for incomplete trials

In [ ]:
import sqlite3
con = sqlite3.connect(f'{study_name}.db')
cursor = con.cursor()

cursor.execute("SELECT * FROM trials") #show all values in trials table
print(cursor.fetchall())

cursor.execute("DELETE from trials WHERE value is null")
cursor.execute("SELECT value FROM trials") #prove u removed the None values
print(cursor.fetchall())

con.commit() #save changes
cursor.close() #close connecton